# **II. Read and write with pandas**

In [0]:
### Steps for use with colab
# First step to mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive
# Clone Pyladies repo 
#! git clone --recursive https://github.com/pyladiesams/Pandas-advanced-nov2019.git
# Install requirements
! pip install pandas==0.25.3
import pandas as pd
# Move into repo
%cd /content/drive/My\ Drive/Pandas-advanced-nov2019/workshop/

# **Introduction**

Pandas has two types of labeled data structures especially adapted for data analysis:
- series (1-dimensional)
- dataframes (2-dimensional)

## Supported formats

This module comes with several I/O functions to read/write data from/into a pandas object. 

Here is an exhaustive list of these functions showing the different supported formats : 
- Reader / Writer
- read_csv / to_csv
- read_json / to_json
- read_html / to_html
- read_clipboard / to_clipboard
- read_excel / to_excel
- read_hdf / to_hdf
- read_feather / to_feather
- read_parquet / to_parquet
- read_msgpack / to_msgpack
- read_stata / to_stata 
- read_pickle / to_pickle
- read_sql / to_sql
- read_gbq / to_gbq (Google Big Query)



Documentation about how to use all these functions is available here:

https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

## Using csv reader with default parameterization
Use the read_csv function to load the following file :  airquality_default.csv

This function will automatically interprete the data and load it into a dataframe. It has a lot of parameters that can be customized but here only default values will be applied that may or may not fit the format of the input file. 





In [0]:
# Read airquality_default.csv
df=pd.read_csv('data/airquality_default.csv')
df.head(5)

This file is completely standard and had been correctly decoded by the reader.

Now try to load the following file :
airquality.csv

In [0]:
# Read airquality.csv
df=pd.read_csv('data/airquality.csv')
df.head(5)

All the data has been loaded as 2 columns because the delimiter is not correct.


# **Customize csv reader**

In this section we will play with some paramterers to allow you to fully customize the reader.



## Delimiter
Default delimiter is ;

Two parameters can be passed to the function to set up the delimiter:  
- delimiter
- sep

Play with the delimiter parameters to correctly decode airquality.csv

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';')
df.head(5)

## Header
Try to add the parameter header and set it to:
- 0 (default is to auto infer the column names)
- None (does not interprete the column names)

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';', header=0)
df.head(5)

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',header=None)
df.head(5)


With header set to None, no column names have been inferred but they can be overwritten.

## Overwrite column names
Use those two paramters to overwrite the column names :
- header : None
- names : list of strings

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',header=None, names=['time','location','component','concentration','risk_index'])
df.head(5)

There is still one issue here: the first row contains the header and has been decoded as data. 

The first row should be skipped.

## Skip lines
Several parameters exist to skip lines:
- skip_blank_lines (default is True)
- comment (give a string and if a row starts with this specified string the row will be ignored)
- error_bad_lines (True : turned on by default) 
- skiprows (index or list of index of the rows that should be skipped, supports also callable functions)

Play with these parameters to properly load airquality.csv

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',skiprows=1,header=None, names=['time','location','component','concentration','risk_index'])
df.head(5)

The location column does not make sense here since all the data comes from the same sensor. 

Drop the location column with drop function

In [0]:
df.drop(columns='location',inplace=True)

In [0]:
df.dtypes

## Read a subset of columns
Dropping columns can be done in another way directly from the reader.

Use the following parameter to read only a subset of columns :
- usecols (list of index or list of string names)

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',skiprows=1,header=None, usecols=[0,2,3,4],names=['time','component','concentration','risk_index'])
df.head(5)


## Large files : read in chunks
If you have to read very large files you can use the following parameter :
- chunksize

This will split the data into smaller chunks of which you can specify the size and output the data as an iterator.

Let's pretend we have a huge file here and try to load it with chunks of 10000 rows each.

In [0]:
# Read airquality.csv in chunks
df=pd.read_csv('data/airquality.csv',sep=';',chunksize=10000,skiprows=1,header=None, usecols=[0,2,3,4],names=['time','component','concentration','risk_index'])


Let's have a look at the schema of each chunk of data


In [0]:
for chunk in df:
  print(chunk.dtypes)

We can see that the last column has been decoded as integer.


Update a slice of this column (5:15) for this chunk and set it to None

Tip : use iloc

Then print the schema again, what happened ? 

In [0]:
chunk.iloc[5:15,3]=None
chunk.dtypes

This column is now set up as float because null values are not supported by int64 type. 

Since a recent upgrade, a new type Int64 has been introduced to support int and null values.

You can try to convert this column with astype function successively to int64 and Int64 to check this out.

In [0]:
chunk['risk_index']=chunk['risk_index'].astype('int64')


In [0]:
chunk['risk_index']=chunk['risk_index'].astype('Int64')
chunk.dtypes


# **Data types**

The most commonly used types available in pandas data structures :
- int64 / Int64
- float64
- datetime64
- bool
- object (mixed types)

## Specify the schema
Why ?
- avoid misattribution of the types by the reader -> this can lead to broken ETL pipelines
- when dealing with large data files this improves the performance

How ?
- usecols 
- dtype

Load the airquality.csv file by playing with usecols, dtype and decimal parameters to load the data with the following schema:
- time : object
- component : object
- concentration : float64
- risk_index : Int64

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',decimal=',',skiprows=1,header=None, usecols=[0,2,3,4],names=['time','component','concentration','risk_index'],dtype={0:'object',2:'object',3:'float64',4:'Int64'})
df.dtypes

## Converters
Data can be converted on the fly while reading the file by passing the parameter:
- converters : dict of functions for converting values in certain columns. Keys can either be integers or column labels.

Warning : if both converters and dtype are passed for the same column to the reader converter will be applied and dtype will be ignored

Write a function to convert the column time to datetime

In [0]:
def convert_to_datetime(col, fmt = "%Y-%m-%d %H:%M:%S"):
    return pd.to_datetime(col, format=fmt, utc=True)

Write another function to apply a conversion from micrograms/m3 to ppb for carbon monoxide (CO) 

The conversion factor can be found in the following pdf:

    https://www2.dmu.dk/AtmosphericEnvironment/Expost/database/docs/PPM_conversion.pdf

In [0]:
# Apply conversion micrograms/m3 to ppb for carbon monoxide
def co_to_ppb(col):
    return float(col)/1.145

In [0]:
# Test your function by using apply to your column and check it 
df['concentration']=df['concentration'].apply(co_to_ppb)

Then use your functions with the converters parameter to read airquality.csv

In [0]:
df=pd.read_csv('data/airquality.csv',sep=';',decimal=',',skiprows=1,header=None, usecols=[0,2,3,4],names=['time','component','concentration','risk_index'],dtype={0:'object',2:'object',3:'float64',4:'Int64'},converters ={0:convert_to_datetime,4:co_to_ppb})
df.dtypes

## Booleans
- true_values : list of strings to be decoded as True
- false_values : list of strings to be decoded as False

## Missing data
- na_values (default list [ ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’])
- keep_default_na (keep the default list and append what you specified as na_values)
- na_filter (filter out na values default is **False**)

## Bad lines
Data providers can make mistakes. Therefore it is helpful to have a process in place that won't break in case of bad lines
- error_bad_lines 
    - default is True, i.e. bad lines will generate an error no dataframe is returned)
    - False bad lines will be dropped from the final dataframe

Read the file airquality_bad.csv by playing with error_bad_lines

In [0]:
df=pd.read_csv('data/airquality_bad.csv',error_bad_lines=False)
df.head(5)


## Encoding
It is always good practice to have an understanding of the encoding of your data files. Because a mismatch between encoding and decoding can lead to errors.

- encoding

An exhaustive list of the standard python encodings can be found here:
https://docs.python.org/3/library/codecs.html#standard-encodings

Read airquality_encode.csv with utf-8 encoding what happens ?

In [0]:
df=pd.read_csv('data/airquality_encode.csv',encoding='utf-8',error_bad_lines=False)
df.head(5)


Try another encoding and fix this error.

In [0]:
df=pd.read_csv('data/airquality_encode.csv',encoding='utf-16',error_bad_lines=False)
df.head(5)
df.dtypes

# Exercises




## Exercise 1:
Load the file airquality_mess.csv in a dataframe having the following schema :
*   time :              datetime64
*   component :         object
*   concentration :     float64
*   risk_index :        int64
*   low_risk :          bool



Tip: you can reuse the converters







In [0]:
df=pd.read_csv(
    'data/airquality_mess.csv',
    sep=';',
    dtype={3:'Int64',4:'bool'},
    converters={0:convert_to_datetime,2:co_to_ppb},
    true_values=['t'],
    false_values=['f','N-A'],
    encoding='utf-8'
)
df.head(5)
df.dtypes


## Exercise 2:
As previsouly in exercice 1 but load only the data about carbon monoxide component (CO)

Tip : write a callable function to select the indices of the rows to be loaded

In [0]:
def keep_co(i):
  if i%7 ==1 or i==1:
    return False
  else:
    return True

df=pd.read_csv('data/airquality_mess.csv',sep=';',header=None,names=['time','component','concentration','risk_index','low_index'],skiprows=keep_co,dtype={3:'Int64',4:'bool'},converters={0:convert_to_datetime,2:co_to_ppb},true_values=['t'],false_values=['f','N-A'],encoding='utf-8')
df.head(5)



## Exercise 3:

* Load the airquality.csv file
* Corrupt the file with some magic (apply, drop ...)
* Save the result in a file airquality_corrupt.FMT (use the format of your choice)
* Let your peer decode it in one shot !

# Regenerate files from airquality.csv

In [0]:
#@Generate default file
df=pd.read_csv('data/airquality.csv', delimiter=';', decimal=','); 
df.to_csv('data/airquality_default.csv',sep=',', decimal='.', index=False)


In [0]:
#@Create airquality_bad.csv with bad lines
df=pd.read_csv('data/airquality.csv',sep=';',usecols=[0,2,3,4], header=None,names=['time','component','concentration','risk_index'], skiprows=1, dtype={4:'Int64'}, decimal=','); df.head(5)
df.to_csv('data/airquality_bad.csv',sep=',', index=False, encoding='utf-8')
df.loc[df['risk_index']<5,'low_risk']='t'
df.loc[df['risk_index']>=5,'low_risk']='f'
df.to_csv('data/airquality_bad.csv',sep=',', mode='a', index=False, encoding='utf-8')


In [0]:
# Create airquality_encode.csv with different encoding
df=pd.read_csv('data/airquality.csv',sep=';',usecols=[0,2,3,4], header=None,names=['time','component','concentration','risk_index'], skiprows=1, dtype={4:'Int64'}, decimal=','); df.head(5)
df.to_csv('data/airquality_encode.csv',sep=',', index=False, encoding='utf-16')

In [0]:
# Create airquality_mess.csv
df=pd.read_csv('data/airquality_encode.csv',encoding='utf-16', error_bad_lines=False); 
df.loc[df['risk_index']<5,'low_risk']='t'
df.loc[(df['risk_index']>=5),'low_risk']='f'
df.loc[df['risk_index']==11,'low_risk']='N-A'
df.loc[df['component']=='O3','risk_index']=None
df.sort_values(by=['time','component'],ascending=True, inplace=True)
df.reset_index(inplace=True,drop=True)
df.head(5)
df.to_csv('data/airquality_mess.csv',sep=';',index=False)
df.head(20)